In [3]:
import pandas as pd
from geopy.distance import geodesic
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
geo = pd.read_csv('data/filtered_RJ_lat_long_data.csv')
final = pd.read_csv('data/final_combined_dataset.csv')

final.isnull().sum()

date                                    0
week                                    0
cases                                   0
population                              0
tempe_min                               0
humidity_max                            0
humidity_avg                            0
humidity_min                            0
temp_avg                                0
temp_max                                0
city                                    0
geocode                                 0
vim                                     0
vim_monthly                             0
precipitation_avg_ordinary_kriging      0
precipitation_max_ordinary_kriging      0
precipitation_avg_regression_kriging    0
precipitation_max_regression_kriging    0
long                                    0
lat                                     0
cases_per_100k                          0
nearby_cases_weighted                   0
dtype: int64

In [25]:
geo['CITY_lower'] = geo['CITY'].str.lower()
final['city_lower'] = final['city'].str.lower()

# Merge datasets using the lowercase city columns
final = pd.merge(final, geo[['CITY_lower', 'LONG', 'LAT']], left_on='city_lower', right_on='CITY_lower', how='left')

final = final.drop(columns=['CITY_lower', 'city_lower'])

# rename 
final = final.rename(columns={'LONG': 'long', 'LAT': 'lat'})


Spatial Aggregation: Create additional features that represent the cases of nearby cities.
For example:
Average cases of neighboring cities.

nearby_cases_weighted: Weighted average of cases in the 3 nearest cities for the same week.

here we will consider cases_per_100k as cases. Because we want to consider the population of the city in the calculation.

In [27]:
final.to_csv('data/final_combined_dataset.csv', index=False)

In [29]:
# First part: Weeks from 201201 to 201701
first_part_weeks = list(range(201201, 201701))

# Update for the first part
first_part_data = data[data['week'].isin(first_part_weeks)]
nearby_weighted_cases_first_part = []

for week in first_part_data['week'].unique():
    weekly_data = first_part_data[first_part_data['week'] == week]
    print(f'Processing week {week}')
    for _, row in weekly_data.iterrows():
        weighted_cases = calculate_weighted_cases(row, weekly_data)
        nearby_weighted_cases_first_part.append(weighted_cases)

data.loc[data['week'].isin(first_part_weeks), 'nearby_cases_weighted'] = nearby_weighted_cases_first_part

# Save the updated data back to CSV
data.to_csv('data/final_combined_dataset.csv', index=False)


In [30]:
data.head()